In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re


In [2]:
csv_folder = r"/Users/qialiang/Documents/Studium/DM_Dataset"
csv_path = os.path.join(csv_folder, "merged_Post_with_votes_comments_and_users.csv")
df = pd.read_csv(csv_path)

/var/folders/mg/7s9vdy_90bz_8g1dvhh46rzr0000gn/T/ipykernel_11898/2681688271.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

# 1. 清洗标签
df['TagList'] = df['posts.Tags'].str.strip('|').str.split('|')

# 2. 文本向量化（TF-IDF）
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
X_tfidf = tfidf.fit_transform(df['posts.CleanBodyNoMath'])

# 3. 建立 NMF 模型
n_topics = 50  # 可以调参
nmf = NMF(n_components=n_topics, random_state=42)
W = nmf.fit_transform(X_tfidf)  # shape: [n_samples, n_topics]
H = nmf.components_             # shape: [n_topics, n_words]

# 4. 转换标签为矩阵
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['TagList'])  # shape: [n_samples, n_tags]

KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(LogisticRegression())
model.fit(W, Y)

# 预测
pred_proba = model.predict_proba(W[0:1])
pred_tags = mlb.inverse_transform(pred_proba > 0.2)